In [1]:
import rpy2
import rpy2.robjects as robjects

print(rpy2.__version__)

3.5.11


In [2]:
script = '''
library(bnlearn)
library(gRain)
'''

_ = robjects.r(script)

R[write to console]: Loading required package: gRbase

R[write to console]: 
Attaching package: ‘gRbase’


R[write to console]: The following objects are masked from ‘package:bnlearn’:

    ancestors, children, nodes, parents




In [3]:
v = robjects.r("v <- readRDS('./rds/huang-v.rds')")
p = robjects.r("p <- readRDS('./rds/huang-p.rds')")
g = robjects.r("g <- readRDS('./rds/huang-g.rds')")
b = robjects.r("b <- readRDS('./rds/huang-b.rds')")
t = robjects.r("t <- readRDS('./rds/huang-t.rds')")

In [4]:
n2v = {_item[0]: [_i for _i in _item[1]] for _item in v.items()}
n2v

{'A': ['on', 'off'],
 'B': ['on', 'off'],
 'C': ['on', 'off'],
 'D': ['on', 'off'],
 'E': ['on', 'off'],
 'F': ['on', 'off'],
 'G': ['on', 'off'],
 'H': ['on', 'off']}

In [5]:
def query(script='querygrain(t, nodes=nodes(g))'):
    print(f'{script=}')
    e = robjects.r(script)
    _n = [_n for _n in e.names]
    _p = [[_i for _i in _e] for _e in e]
    
    return {name: {_val: _prob for _val, _prob in zip(n2v[name], probs)} 
            for name, probs in zip(_n, _p)}

def get_posteriors(e=None):
    if e is None or len(e) == 0:
        return query()
    
    keys = e.keys()
    
    nodes = ','.join([f'"{k}"' for k in keys])
    nodes = f'c({nodes})'
    
    states = ','.join([f'"{e[k]}"' for k in keys])
    states = f'c({states})'
    
    q = f'querygrain(setEvidence(t, nodes={nodes}, states={states}), nodes=nodes(g))'
    q = query(q)
    
    p = {_n: {_val: 1.0 if _val == _v else 0.0 for _val in n2v[_n]} 
         for _n, _v in e.items()}
    
    return {**p, **q}

In [6]:
get_posteriors()

script='querygrain(t, nodes=nodes(g))'


{'A': {'on': 0.5, 'off': 0.5},
 'B': {'on': 0.45000000000000007, 'off': 0.55},
 'C': {'on': 0.4499999999999999, 'off': 0.55},
 'D': {'on': 0.6799999999999999, 'off': 0.32},
 'E': {'on': 0.4650000000000001, 'off': 0.535},
 'F': {'on': 0.17630600000000002, 'off': 0.823694},
 'G': {'on': 0.415, 'off': 0.5850000000000001},
 'H': {'on': 0.8231, 'off': 0.17689999999999997}}

In [7]:
get_posteriors({'A': 'on'})

script='querygrain(setEvidence(t, nodes=c("A"), states=c("on")), nodes=nodes(g))'


{'A': {'on': 1.0, 'off': 0.0},
 'B': {'on': 0.5, 'off': 0.5},
 'C': {'on': 0.7000000000000001, 'off': 0.30000000000000004},
 'D': {'on': 0.7, 'off': 0.3},
 'E': {'on': 0.39, 'off': 0.61},
 'F': {'on': 0.18934, 'off': 0.8106599999999999},
 'G': {'on': 0.59, 'off': 0.41},
 'H': {'on': 0.7826, 'off': 0.2174}}

In [8]:
get_posteriors({'A': 'on', 'B': 'on'})

script='querygrain(setEvidence(t, nodes=c("A","B"), states=c("on","on")), nodes=nodes(g))'


{'A': {'on': 1.0, 'off': 0.0},
 'B': {'on': 1.0, 'off': 0.0},
 'C': {'on': 0.7, 'off': 0.30000000000000004},
 'D': {'on': 0.9, 'off': 0.1},
 'E': {'on': 0.39, 'off': 0.61},
 'F': {'on': 0.06978000000000001, 'off': 0.93022},
 'G': {'on': 0.59, 'off': 0.41},
 'H': {'on': 0.7826, 'off': 0.2174}}

In [9]:
get_posteriors({'A': 'on', 'B': 'off'})

script='querygrain(setEvidence(t, nodes=c("A","B"), states=c("on","off")), nodes=nodes(g))'


{'A': {'on': 1.0, 'off': 0.0},
 'B': {'on': 0.0, 'off': 1.0},
 'C': {'on': 0.7, 'off': 0.3},
 'D': {'on': 0.5, 'off': 0.5},
 'E': {'on': 0.39000000000000007, 'off': 0.61},
 'F': {'on': 0.3089, 'off': 0.6911},
 'G': {'on': 0.5900000000000001, 'off': 0.41},
 'H': {'on': 0.7826, 'off': 0.2174}}